1.словарь - count 
2. по расстоянию ливенштейна 10

In [1]:
import pandas as pd
data = pd.read_csv('data_ifax/index.csv',  # Это то, куда вы скачали файл
                       sep=',')
data.head(10)

,Id,Url,Title
0,266EFC602F3F3E3209ADD9E475AB2B52,http://www.interfax.ru/moscow/582336,"Огонь полностью охватил строительный рынок ""Си..."
1,E336BF4DC7A5AF06FC14306EDEF2028C,http://www.interfax.ru/russia/582326,В Дагестане началась проверка из-за задержания...
2,FBAE7E7AC5B9C81AFEFFAE4B2F8DB12B,http://www.interfax.ru/russia/582315,В Госдуме анонсировали принятие закона о запре...
3,086D54F89C5833F7AF9CFFE2709582F5,http://www.interfax.ru/world/582312,Премьер Испании исключил отделение Каталонии о...
4,62C3A495624B49D7AA38767D26A5CCFC,http://www.interfax.ru/russia/582298,В Петербурге завели дело после смерти женщины ...
5,5F277BE8F55F76F68ADF31830A7280B5,http://www.interfax.ru/world/582216,Полиция Дании обнаружила части тела пропавшей ...
6,FE188E1820FF5ED08BB3AE0FE8D1C5F3,http://www.interfax.ru/russia/582375,Житель Красноярска предстанет перед судом за р...
7,41009F19FE4F209F1EF55DFD7A93A849,http://www.interfax.ru/russia/582374,Медведев предложил поделиться с Алжиром технол...
8,FE383F9330C83173FE09F741B720D0BE,http://www.interfax.ru/russia/582373,Житель Приамурья приговорен к 22 годам строгог...
9,018CABDC8CAABBE7AC8A4E32E24196D9,http://www.interfax.ru/russia/582371,Площадь лесных пожаров в Приморье за сутки выр...


In [8]:
data['Id'].shape

(34822,)

In [107]:
import os

In [109]:
directory = 'data_ifax/'
file = os.listdir(directory)
slovar_imen_file = []
for x in file:
    #print(directory+file[1]+'/')
    if x != 'index.csv':
        new_f = os.listdir(directory+x+'/')
        #f = open(directory+file)
        #print(new_f[1])
        slovar_imen_file.append(directory+x+'/'+new_f[1])
print(slovar_imen_file[:10])
print(len(slovar_imen_file))

['data_ifax/0000469F976CDC26F/CD8903D47E0BEED.txt', 'data_ifax/00014CA54E1083C9E/A8D9864C98ABF58.txt', 'data_ifax/0003B7CE1437FDE32/191A4E229010A73.txt', 'data_ifax/0003C742579B14AD9/DA72FD2A12650E1.txt', 'data_ifax/0004CAEE762261483/562E7792A6B7C99.txt', 'data_ifax/0005D704757007CC6/DDDA26B32618C55.txt', 'data_ifax/000696B032E397B42/3408ABB02C4E363.txt', 'data_ifax/0009A352E003C569D/4CBF0956E903298.txt', 'data_ifax/000A15C54A91F7D0C/2DA7325CE6E53C8.txt', 'data_ifax/000A32031EFD0FC90/C301179430A78A2.txt']
34822


In [106]:
s = 'Мосва'
chast_len = len(s) % 3
s[chast_len*2:]

'а'

In [110]:
def clear_world_symbols(massiv):
    global slovar_symbols, slovar_slov, tri_geamm
    #print(massiv)
    for x in massiv:
        for slo in x.strip().split():
            slo = slo.strip().lower()
            #print(slo)
            if slovar_slov.get(slo) == None:
                slovar_slov[slo] = 1
            else:
                slovar_slov[slo] += 1
                
            chast_len = len(slo) % 3 
            slovar_gr = [slo[:chast_len],slo[chast_len:chast_len*2],slo[chast_len*2:]]
            for tr in slovar_gr:
                if tri_geamm.get(tr) == None:
                    tri_geamm[tr] = [slo]
                else:
                    tri_geamm[tr].append(slo)
            
                for y in slo:
                    if slovar_symbols.get(y) == None:
                        slovar_symbols[y] = [slo]
                    else:
                        slovar_symbols[y].append(slo)

In [132]:
slovar_symbols ={}
slovar_slov = {} # slovo - count
tri_geamm = {}

for file in slovar_imen_file[:5000]:#[:10]:#[:5000]:
    #print(file)
    f = open(file, 'r', encoding = 'utf-8')
    clear_world_symbols([x for x in f])

In [133]:
def distance(a, b):
    "Calculates the Levenshtein distance between a and b."
    n, m = len(a), len(b)
    if n > m:
        # Make sure n <= m, to use O(min(n, m)) space
        a, b = b, a
        n, m = m, n

    current_row = range(n + 1) # Keep current and previous row, not entire matrix
    for i in range(1, m + 1):
        previous_row, current_row = current_row, [i] + [0] * n
        for j in range(1, n + 1):
            add, delete, change = previous_row[j] + 1, current_row[j - 1] + 1, previous_row[j - 1]
            if a[j - 1] != b[i - 1]:
                change += 1
            current_row[j] = min(add, delete, change)

    return current_row[n]

In [138]:
def check_levi(slovo):
    global slovar_slov
    slovo = slovo.strip().lower()
    position = []
    slovar_slov_position = {}
    if slovar_slov.get(slovo) == None:
        for x in slovar_slov:
            save_otv = distance(slovo,x)
            if slovar_slov_position.get(save_otv) == None:
                slovar_slov_position[save_otv] = [x]
            else:
                slovar_slov_position[save_otv].append(x)
                
        for x in slovar_slov_position:
            position.append(x)
        position = sorted(position)
        otvet = []
        l = 0
        while len(otvet) < 10:
            otvet += slovar_slov_position[position[l]]
            l+= 1
        for x in otvet[:10]:
            print(x,':(',slovar_slov[x],',',distance(slovo,x),')')
        #print(otvet[:10])
    else:
        print(slovo,':',slovar_slov[slovo])

In [150]:
def check_levi_index(slovo):
    global tri_geamm, slovar_slov
    slovo = slovo.strip().lower()
    slovat = []
    
    chast_len = len(slovo) % 3 
    slovar_gr = [slovo[:chast_len],slovo[chast_len:chast_len*2],slovo[chast_len*2:]]
    for tr in slovar_gr:
        if tri_geamm.get(tr) != None:
                slovat += tri_geamm[tr]
    #print (slovat)
    
    position = []
    slovar_slov_position = {}
    for x in slovat:
        save_otv = distance(slovo,x)
        if slovar_slov_position.get(save_otv) == None:
                slovar_slov_position[save_otv] = [x]
        else:
                slovar_slov_position[save_otv].append(x)    
    #print(slovar_slov_position) 
    
    for x in slovar_slov_position:
        position.append(x)
    position = sorted(position)
    #print(position)
    
    otvet = []
    l = 0
    while len(otvet) < 10:
        otvet += slovar_slov_position[position[l]]
        l+= 1
        #print(slovar_slov_position[position[l-1]])
        #print(slovar_slov_position[position[l]])    
    for x in otvet[:10]:
        print(x,':(',slovar_slov[x],',',distance(slovo,x),')')
    print(otvet[:10])

# Расстояние левинштейна стандарт

In [ ]:
check_levi('Мосва')

# По индексу

In [ ]:
check_levi_index('Москва')